In [ ]:
!pip install datasets spacy transformers tqdm langchain-core
!pip install langchain-groq
!python -m spacy download en_core_web_sm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.9/124.9 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 103.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import os
import pandas as pd
import spacy
import random
import time
from tqdm import tqdm
from datasets import load_dataset
from langchain_groq import ChatGroq
from langchain.schema import HumanMessage, SystemMessage

In [ ]:
nlp = spacy.load("en_core_web_sm")
print("Loading WikiMIA-24 dataset...")
dataset = load_dataset("wjfu99/WikiMIA-24")
print("Dataset loaded successfully!")


df = pd.DataFrame(dataset["WikiMIA_length64"]).head(50)
print(f"Selected {len(df)} samples from WikiMIA_length64")

Loading WikiMIA-24 dataset...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/2.23k [00:00<?, ?B/s]

(…)-00000-of-00001-b37e6e80280f9a92.parquet:   0%|          | 0.00/144k [00:00<?, ?B/s]

(…)-00000-of-00001-c1a351cf2dd00067.parquet:   0%|          | 0.00/298k [00:00<?, ?B/s]

...

Generating WikiMIA_length32 split:   0%|          | 0/1558 [00:00<?, ? examples/s]

Generating WikiMIA_length64 split:   0%|          | 0/1396 [00:00<?, ? examples/s]

Generating WikiMIA_length128 split:   0%|          | 0/572 [00:00<?, ? examples/s]

Generating WikiMIA_length256 split:   0%|          | 0/186 [00:00<?, ? examples/s]

Dataset loaded successfully!
Selected 50 samples from WikiMIA_length64


In [ ]:
df = pd.DataFrame(dataset["WikiMIA_length64"]).head(50)
print(f"Selected {len(df)} samples from WikiMIA_length64")

Selected 50 samples from WikiMIA_length64


In [ ]:
GROQ_API_KEY = os.environ.get("GROQ_API_KEY", "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")
if not GROQ_API_KEY:
    GROQ_API_KEY = input("Enter your Groq API key: ")
os.environ["GROQ_API_KEY"] = GROQ_API_KEY


In [ ]:
groq_model = ChatGroq(model_name="llama3-70b-8192")

In [ ]:
DOMAIN_FUSION_PROMPT = """You are given an input sentence: {input_sentence}. Your goal is to create a perturbed version by fusing the original content while preserving its core meaning. Follow these steps:
1. Dependency Parsing: Analyze the sentence to identify its syntactic structure. Extract key components such as the main verb, subject, object, and important modifiers.
2. Select an Unrelated Domain: Choose a domain that is semantically distinct (e.g., culinary, sports, art).
3. Integrate Domain-Specific Elements: Rewrite the sentence by incorporating terms, analogies, or metaphors from the selected domain. Ensure the original meaning is maintained even as new imagery is introduced.
4. Output: Provide the final, perturbed sentence.
Let's think step by step:
- Identify the core components (e.g., subject, verb, object) of the sentence.
- Choose a domain (e.g., culinary) and list relevant terms or imagery (like "layered pastry" or "ingredients").
- Integrate these elements into the sentence in a natural way so that it retains the original context.

Return ONLY the perturbed sentence, with no explanations."""

LEXICAL_PROMPT = """You are given an input sentence: {input_sentence}. Your task is to introduce subtle noise by modifying roughly 10% of the tokens. Replace selected tokens with rare Unicode variations or synonyms that alter the surface form while preserving the original meaning. Follow these steps:
1. Tokenization: Break the sentence into individual tokens (words and punctuation).
2. Select Tokens for Perturbation: Identify approximately 10% of the tokens that can be safely replaced without changing the sentence's meaning.
3. Apply Perturbations: For each selected token, substitute it with a variant that uses uncommon Unicode characters or a synonym that appears slightly different.
4. Output: Produce the final perturbed sentence.
Let's think step by step:
- Tokenize the sentence into its component parts.
- Randomly choose about 10% of tokens that do not compromise the semantic content.
- Replace each chosen token with a visually distinct variant (e.g., replacing "a" with "ä" or "e" with "ë").
- Reassemble the sentence, ensuring the original meaning remains clear.

Return ONLY the perturbed sentence, with no explanations."""

TEMPORAL_PROMPT = """You are given an input sentence: {input_sentence}. Your objective is to reframe the sentence to evoke a different temporal perspective while preserving its semantic meaning. Follow these steps:
1. Identify Temporal Elements: Examine the sentence to note any contemporary references or phrasing.
2. Reframe the Temporal Context: Alter the phrasing to reflect a non-contemporary perspective. This may include changing the narrative voice, employing past-tense constructions, or adding subtle historical analogies.
3. Enhance with Contextual Nuance: Optionally, introduce elements that hint at a different era (for instance, referencing a bygone style or historical context).
4. Output: Provide the final, temporally shifted sentence.
Let's think step by step:
- Recognize modern expressions or structures in the sentence.
- Convert these into phrasing that suggests an alternative, perhaps older, narrative style.
- Ensure that while the temporal context changes, the core meaning and details of the sentence remain intact.

Return ONLY the perturbed sentence, with no explanations."""

In [ ]:
def generate_perturbation(sentence, prompt_template, max_retries=3):
    """
    Generate a perturbation for a given sentence using LangChain Groq integration
    """
    prompt = prompt_template.format(input_sentence=sentence)

    for attempt in range(max_retries):
        try:

            messages = [
                SystemMessage(content="You are a helpful text perturbation assistant."),
                HumanMessage(content=prompt)
            ]


            response = groq_model.invoke(messages).content.strip().replace("Here is the perturbed sentence:", "").strip()
            return response

        except Exception as e:
            print(f"Error (attempt {attempt+1}/{max_retries}): {str(e)}")
            if attempt < max_retries - 1:
                sleep_time = 2 * (attempt + 1)  # Exponential backoff
                print(f"Retrying in {sleep_time} seconds...")
                time.sleep(sleep_time)
            else:
                return f"Error: Failed after {max_retries} attempts"

    return "Error: Could not generate perturbation"

In [ ]:
def generate_all_perturbations(sentence):
    """
    Generate all three types of perturbations for a given sentence
    """
    perturbations = {}

    print(f"Generating domain fusion perturbation...")
    perturbations["domain_fusion"] = generate_perturbation(sentence, DOMAIN_FUSION_PROMPT)

    print(f"Generating lexical perturbation...")
    perturbations["lexical"] = generate_perturbation(sentence, LEXICAL_PROMPT)

    print(f"Generating temporal perturbation...")
    perturbations["temporal"] = generate_perturbation(sentence, TEMPORAL_PROMPT)

    return perturbations

In [ ]:
input_sentences = df["input"].tolist()


print("Generating perturbations...")
perturbed_data = []
for i, sentence in enumerate(tqdm(input_sentences)):
    print(f"\nProcessing sentence {i+1}/{len(input_sentences)}")
    print(f"Original: {sentence}")

    perturbations = generate_all_perturbations(sentence)


    entry = {
        "original": sentence,
        "original_label": df.iloc[i]["label"],
        "perturbation_type": "PM" if df.iloc[i]["label"] == 1 else "PNM",
        "domain_fusion": perturbations["domain_fusion"],
        "lexical": perturbations["lexical"],
        "temporal": perturbations["temporal"]
    }

    perturbed_data.append(entry)


    print(f"Domain Fusion: {perturbations['domain_fusion']}")
    print(f"Lexical: {perturbations['lexical']}")
    print(f"Temporal: {perturbations['temporal']}")


    time.sleep(1)


Generating perturbations...


  0%|          | 0/50 [00:00<?, ?it/s]


Processing sentence 1/50
Original: The 109th edition of the Paris–Tours cycling classic was held on 11 October 2015. The race was part of the 2015 UCI Europe Tour, ranked as a 1.HC event. Italian rider Matteo Trentin won the race in a three-man sprint with Belgians Tosh Van der Sande and Greg Van Avermaet. At an average speed of 49.64 km/h (30.84 mph), it was the fastest edition
Generating domain fusion perturbation...
Generating lexical perturbation...
Generating temporal perturbation...
Domain Fusion: The 109th edition of the Paris–Tours cycling classic was the pièce de résistance, unfolding on 11 October 2015, with Italian rider Matteo Trentin expertly whisking the competition in a three-man sprint, outpacing Belgians Tosh Van der Sande and Greg Van Avermaet, at an average speed of 49.64 km/h (30.84 mph), the fastest edition yet, a true culinary masterpiece of strategy and endurance.
Lexical: The 109th édition of the Päris–Tours cyclïng classic was held on 11 Octöber 2015. The raçe

  2%|▏         | 1/50 [00:03<02:27,  3.01s/it]


Processing sentence 2/50
Original: The 2024 Scottish Challenge Cup final, also known as the SPFL Trust Trophy final for sponsorship reasons, was a football match on 24 March 2024 between the Cymru Premier club The New Saints and the Scottish Championship side Airdrieonians. It was the 31st final of the Scottish Challenge Cup since it was first organised in 1990 to celebrate the centenary of the now defunct
Generating domain fusion perturbation...
Generating lexical perturbation...
Generating temporal perturbation...
Domain Fusion: The 2024 Scottish Challenge Cup final, a richly layered tournament, was a football match on 24 March 2024 between the Cymru Premier club The New Saints, the main ingredient, and the Scottish Championship side Airdrieonians, the perfect blend, in a recipe for success that was the 31st final of the Scottish Challenge Cup since it was first mixed in 1990 to celebrate the centenary of the now defunct.
Lexical: Thë 2024 Scotïsh Chällënge Cup finäl, alsö known as 

...

 96%|█████████▌| 48/50 [09:49<00:26, 13.36s/it]


Processing sentence 49/50
Original: This is a timeline of major events leading up to, during, and after the 2024 United States presidential election. This will be the first presidential election to be run with population data from the 2020 census. In addition to the dates mandated by the relevant federal laws such as those in the U.S. Constitution and the Electoral Count Act, several milestones have consistently been
Generating domain fusion perturbation...
Generating lexical perturbation...
Generating temporal perturbation...
Domain Fusion: This is a rich tapestry of major events leading up to, during, and after the 2024 United States presidential election, carefully layered like a croquembouche, with population data from the 2020 census serving as the essential ingredient, and in addition to the dates mandated by the relevant federal laws, such as those in the U.S. Constitution and the Electoral Count Act, several milestones have consistently been the icing on the cake.
Lexical: Thï

 98%|█████████▊| 49/50 [10:03<00:13, 13.40s/it]


Processing sentence 50/50
Original: UFC 202: Diaz vs. McGregor 2 was a mixed martial arts event produced by the Ultimate Fighting Championship that was held on August 20, 2016, at the T-Mobile Arena in Paradise, Nevada, part of the Las Vegas metropolitan area. This event was the first one under the full control of the new ownership group consisting of William Morris Endeavor, Silver Lake Partners, Kohlberg Kravis
Generating domain fusion perturbation...
Generating lexical perturbation...
Generating temporal perturbation...
Domain Fusion: UFC 202: Diaz vs. McGregor 2 was a richly layered event produced by the Ultimate Fighting Championship, a culinary mastermind that carefully blended the ingredients of mixed martial arts, holding its serving on August 20, 2016, at the T-Mobile Arena in Paradise, Nevada, part of the Las Vegas metropolitan area, a savory dish under the full control of the new ownership group consisting of William Morris Endeavor, Silver Lake Partners, and Kohlberg Kravi

100%|██████████| 50/50 [10:16<00:00, 12.33s/it]


In [ ]:
output_df = pd.DataFrame(perturbed_data)
output_file = "perturbed_wikimia24_groq.csv"
output_df.to_csv(output_file, index=False)
print(f"CSV file saved: {output_file}")

CSV file saved: perturbed_wikimia24_groq.csv


In [ ]:
print("\nSample of perturbed texts:")
sample_idx = random.randint(0, len(output_df)-1)
print(f"Original: {output_df.iloc[sample_idx]['original']}")
print(f"Domain Fusion: {output_df.iloc[sample_idx]['domain_fusion']}")
print(f"Lexical: {output_df.iloc[sample_idx]['lexical']}")
print(f"Temporal: {output_df.iloc[sample_idx]['temporal']}")


Sample of perturbed texts:
Original: UFC 202: Diaz vs. McGregor 2 was a mixed martial arts event produced by the Ultimate Fighting Championship that was held on August 20, 2016, at the T-Mobile Arena in Paradise, Nevada, part of the Las Vegas metropolitan area. This event was the first one under the full control of the new ownership group consisting of William Morris Endeavor, Silver Lake Partners, Kohlberg Kravis
Domain Fusion: UFC 202: Diaz vs. McGregor 2 was a richly layered event produced by the Ultimate Fighting Championship, a culinary mastermind that carefully blended the ingredients of mixed martial arts, holding its serving on August 20, 2016, at the T-Mobile Arena in Paradise, Nevada, part of the Las Vegas metropolitan area, a savory dish under the full control of the new ownership group consisting of William Morris Endeavor, Silver Lake Partners, and Kohlberg Kravis, the perfect recipe for a thrilling fight.
Lexical: UFC 202: Diäz vs. McGregör 2 wäs a mixed martïal arts eve

In [ ]:
try:
    from google.colab import files
    files.download(output_file)
    print(f"File '{output_file}' is ready for download")
except ImportError:
    print("Not running in Google Colab. File saved locally.")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

File 'perturbed_wikimia24_groq.csv' is ready for download
